In [1]:
import sys

sys.path.append("../../")

%load_ext autoreload
%autoreload 2

In [2]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import hashlib
from ast import literal_eval
from pathlib import Path
from functools import reduce

from src.utils import find_meta_category
from src.feature_extractor import sample_feature_combinations

## Download prerequisite files

Fetch all the results and feature values


In [3]:
# You can get the experiments file here: 01J6KF3JRCATRJQ9CPJTRV5VBM (https://beaker.org/ds/01J6KF3JRCATRJQ9CPJTRV5VBM/details)
!echo "Fetching experiments list..."
!beaker dataset fetch 01J6KF3JRCATRJQ9CPJTRV5VBM --prefix experiments.txt
!echo "Fetching extracted features..."
!mkdir features/
!beaker dataset fetch 01J6KF3JRCATRJQ9CPJTRV5VBM --prefix features/ 
#!beaker dataset fetch 01J6KFVCRCTYHCZDR0XNK0G9HT --prefix features/
!echo "Fetching helpsteer2 dataset"
!beaker dataset fetch 01J6KBM2VCM9EQ7MER26VBXCCM
!echo "Collating all evaluation results"
%run ../../scripts/fetch_evals_rewardbench.py --output_file results.csv --gpt4_threshold_score 0.658 --experiment_prefix rm-eval-helpsteer2 --experiments_file experiments.txt

Fetching experiments list...
Files: 0          ⠋  
Bytes: 0 B        ⠋  
Files: 1          ⠙  
Bytes: 73.77 KiB  ⠙  
Files: 1          ✔  
Bytes: 73.77 KiB  ✔  
Completed in 100ms: 494.5 KiB/s, 7 files/s
Fetching extracted features...
mkdir: features/: File exists
Files: 0          ⠋  
Bytes: 0 B        ⠋  
Files: 2          ⠙  
Bytes: 75.1 MiB   ⠙  
Files: 8          ⠹  
Bytes: 301.6 MiB  ⠹  
Files: 14         ⠸  
Bytes: 527 MiB    ⠸  
Files: 16         ⠼  
Bytes: 602.9 MiB  ⠼  
Files: 16         ✔  
Bytes: 602.9 MiB  ✔  
Completed in 500ms: 1.131 GiB/s, 31 files/s
Fetching helpsteer2 dataset
Files: 0          ⠋  
Bytes: 0 B        ⠋  
Files: 1          ⠙  
Bytes: 70.58 MiB  ⠙  
Files: 1          ✔  
Bytes: 70.58 MiB  ✔  
Files: 1          ✔  
Bytes: 70.58 MiB  ✔  
Completed in 100ms: 361 MiB/s, 5 files/s
Collating all evaluation results
2024-09-03 09:55:15 - INFO - root - Logged-in as ljm (ljm@allenai.org)
2024-09-03 09:55:16 - INFO - root - Found 249 experiments that match 'rm-eval-

100%|██████████| 249/249 [00:31<00:00,  7.98it/s]

2024-09-03 09:55:47 - INFO - root - Computing category scores...
2024-09-03 09:55:47 - INFO - root - Deriving features from the experiments file: experiments.txt
2024-09-03 09:55:48 - INFO - root - Will attempt merge via feature hash
2024-09-03 09:55:48 - INFO - root - Creating labels in column 'label' with GPT-4 threshold '0.658'
2024-09-03 09:55:48 - INFO - root - Saving 107 results to results.csv
2024-09-03 09:55:48 - INFO - root - Saved on results.csv


Collate feature set for all instances


In [4]:
LEXICAL_FEATS_PATH = Path("features")
DATASET_PATH = Path("helpsteer2_human_vs_gpt4_weighted_for_llama.jsonl")


def get_dataset_features(
    feature_path=LEXICAL_FEATS_PATH, dataset_path=DATASET_PATH
) -> "pd.DataFrame":
    lexical_features = [
        "rouge",
        "bertscore",
        "bertscore_length",
        "entity_sim",
        "cosine_sim",
        "prompt_len",
        "len_longer",
        "len_shorter",
        "token_len_difference",
    ]
    lexical_feature_files = [
        file
        for file in feature_path.glob("*.jsonl")
        if any(file.stem in feat for feat in lexical_features)
    ]
    lexical_feats_df = reduce(
        lambda left, right: left.merge(
            right, on=["id", "prompt", "completion_a", "completion_b"], how="outer"
        ),
        [pd.read_json(file, lines=True) for file in lexical_feature_files],
    )

    df = pd.read_json(dataset_path, lines=True).rename(columns={"prompt_hash": "id"})
    finaldf = df.merge(lexical_feats_df, how="left", on="id").drop(
        columns=["prompt", "completion_a", "completion_b"]
    )

    # Hacky way for token_len_difference
    finaldf = finaldf.rename(columns={"token_len_diff": "token_len_difference"})
    return finaldf

In [5]:
results_df = pd.read_csv("results.csv").dropna()
features_df = get_dataset_features()
print(len(results_df))

107


## Get proportion of instances that fulfill the conditions

1. For each row, get features that were activated
2. Then for each activated feature, we get the proportion by looking at the feature dataframe.
3. The proportion is computed as: `number_of_instance_that_fulfill_a_single_condition` / `total_number_of_instances`


In [6]:
# Inspect nan columns
rows_with_nan = features_df[features_df.isna().any(axis=1)]
nan_columns = rows_with_nan.columns[rows_with_nan.isna().any()]
df_nan_columns = rows_with_nan[nan_columns]
df_nan_columns

,expertise_level,format_constraints
289,None,[]
1317,expert domain knowledge,None
4613,basic domain knowledge,None
4734,general public,None


So what you're going to do instead, is to take the binary_cols, and then for each element of that binary_cols, you compute the "weight"


In [7]:
def compute_instances(feat: str, features_df: "pd.DataFrame") -> float:
    total = len(features_df)
    lexical_features = [
        "rouge",
        "bertscore",
        "bertscore_length",
        "entity_sim",
        "cosine_sim",
        "prompt_len",
        "len_longer",
        "len_shorter",
        "token_len_difference",
    ]

    if feat.split("__")[0] in lexical_features:
        feat_name, value = feat.split("__")
        min_val_str, max_val_str = value.split("|")
        min_val, max_val = float(min_val_str.split("=")[1]), float(
            max_val_str.split("=")[1]
        )
        return features_df[feat_name].between(min_val, max_val).mean()
    else:
        # Parse the feature
        feat_name, value = feat.split("=")
        meta_category = find_meta_category(feat_name)
        if meta_category == "scalar":
            v = value.replace("_", " ")
            return features_df[feat_name].value_counts().get(v) / total
        elif meta_category == "closed_set":
            v = value.replace("_", " ")
            list_of_values = features_df[feat_name].tolist()
            return sum([1 if v in listval else 0 for listval in list_of_values]) / total
        elif meta_category == "open_set":
            list_of_values = features_df[feat_name].tolist()
            return sum([1 if listval else 0 for listval in list_of_values]) / total

        return find_meta_category(feat_name)


feats = results_df.columns[results_df.isin([0, 1]).all()]  # get binary columns
feat_map = {
    feat: compute_instances(feat, features_df) for feat in feats if feat != "label"
}

ratio_df = results_df.apply(
    lambda row: row.map(lambda x: feat_map.get(row.name, 1) if x == 1 else x)
)

# Regressor training


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

## Train LinearRegressor


In [10]:
polyfit = False
binary = False

X = ratio_df[list(feat_map.keys())]
y = ratio_df["Overall"]
if binary:
    X = (X > 0).astype(int)

if polyfit:
    poly = PolynomialFeatures(degree=2, include_bias=False)
    X_poly = poly.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X_poly, y, test_size=0.2, random_state=42
    )
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )


print(f"Train size: {len(X_train)}, test size: {len(X_test)}")

model = LinearRegression()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
# print(f"Feature names: {poly.get_feature_names_out(X.columns)}")
print(f"Mean Squared Error: {mse}")
print(f"Coeeficients: {model.coef_}")
print(f"Intercept: {model.intercept_}")

Train size: 85, test size: 22
Mean Squared Error: 0.0022159736395070202
Coeeficients: [-8.95416072e-01 -8.92760687e-04 -4.73834606e-02 -2.99464142e-02
 -5.43166946e-02  3.21880393e-01 -1.30151400e-01  3.66891162e-02
 -5.95998128e-01 -1.01920883e-02 -3.67576070e-04  6.07554303e-03
 -1.00533546e-01 -9.17860943e-02 -3.62468854e-02 -5.83339936e-01
  6.87920031e-02 -6.81474743e-02 -9.38455549e-03 -3.70459219e-12
 -9.58788604e-13  2.38742359e-12  6.19948537e-13  4.41380266e-12
 -3.52555108e+00 -2.59763656e-01 -3.95659099e-01 -5.21415731e-02
 -5.56829383e-01  4.77395901e-14  6.21724894e-15 -5.69258456e+00
 -4.68892150e-02 -7.10700986e-03 -5.38287762e-01 -6.13251413e+00
 -1.57115312e+00  3.55271368e-15  1.38493507e-02  2.82035974e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  7.80971634e-01 -4.65536115e+00
 -1.04643766e+00  0.00000000e+00 -2.24230023e+01  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -4.34303417e-01  0.00000000e+00
  0.

In [11]:
if not polyfit:
    feature_importance = pd.DataFrame(
        {"Feature": X.columns, "Coefficient": model.coef_}
    )

    # Calculate absolute importance for easier comparison
    feature_importance["Absolute_Coefficient"] = np.abs(
        feature_importance["Coefficient"]
    )

    # Sort by absolute coefficient value
    feature_importance = feature_importance.sort_values(
        by="Absolute_Coefficient", ascending=False
    )
    feature_importance.head(10)
else:
    print(
        "Feature importance is not possible with polynomial features (hard to interpret)"
    )

## Simulation


In [12]:
from tqdm import tqdm_notebook

In [13]:
_, combinations = sample_feature_combinations(
    meta_analyzer_n_samples=2000, max_number=10
)

10it [00:00, 81442.80it/s]
45it [00:00, 100717.01it/s]
120it [00:00, 79262.44it/s]
210it [00:00, 64475.80it/s]
252it [00:00, 47024.27it/s]
210it [00:00, 41576.77it/s]
120it [00:00, 37244.08it/s]
45it [00:00, 32677.23it/s]
10it [00:00, 19963.37it/s]
1it [00:00, 13315.25it/s]

2024-09-03 09:58:04 - INFO - root - Adding meta analyzer features



10it [00:00, 119156.36it/s]
45it [00:00, 100663.30it/s]
120it [00:00, 79676.50it/s]
210it [00:00, 60866.83it/s]
252it [00:00, 51829.78it/s]
210it [00:00, 44099.73it/s]
120it [00:00, 37679.03it/s]
45it [00:00, 32424.61it/s]
10it [00:00, 22832.36it/s]
1it [00:00, 11214.72it/s]


In [14]:
sim_df = pd.DataFrame(0, index=np.arange(len(combinations)), columns=X.columns)
for idx, combination in tqdm_notebook(enumerate(combinations), total=len(combinations)):
    activated_feats = []
    for feat in combination:
        if "analyzer" in feat:
            feature_name_str, value_str = feat.split("::")[1].split("|")
            feature_name, value = (
                feature_name_str.split("=")[-1],
                value_str.split("=")[-1],
            )
            activated_feats.append(f"{feature_name}={value}")
        else:
            activated_feats.append(feat.replace("::", "__"))
    sim_df.loc[idx, activated_feats] = 1
sim_df = sim_df.apply(
    lambda row: row.map(lambda x: feat_map.get(row.name, 1) if x == 1 else x)
).dropna(axis=1, how="any")

/var/folders/hx/gk2rs0792pn5p8hkj4nkhdm80000gp/T/ipykernel_66313/4023816742.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, combination in tqdm_notebook(enumerate(combinations), total=len(combinations)):


  0%|          | 0/4069 [00:00<?, ?it/s]

In [15]:
sim_results = sim_df.copy(deep=True)
sim_results["activated_features"] = sim_results.apply(
    lambda row: [col for col in sim_results.columns if row[col] != 0], axis=1
)
sim_results["pred"] = model.predict(sim_df)
sim_results = sim_results.sort_values(by="pred", ascending=False).reset_index(drop=True)
sim_results[["activated_features", "pred"]].head(20)

,activated_features,pred
0,"[complexity_of_intents=complex, safety_concern...",0.780752
1,"[complexity_of_intents=complex, format_constra...",0.774561
2,"[complexity_of_intents=complex, safety_concern...",0.769075
3,"[complexity_of_intents=simple, languages=Engli...",0.767113
4,"[complexity_of_intents=simple, safety_concern=...",0.764810
5,"[complexity_of_intents=simple, format_constrai...",0.762472
6,"[complexity_of_intents=complex, languages=Engl...",0.759702
7,"[complexity_of_intents=complex, languages=Engl...",0.759702
8,"[complexity_of_intents=complex, languages=Engl...",0.759702
9,"[complexity_of_intents=simple, languages=Engli...",0.759199


In [25]:
top_n = 100
human_score = 0.715
better_than_humans = sim_results[sim_results["pred"] > human_score]
top_combinations = better_than_humans.activated_features.head(top_n).to_list()
print(top_combinations)

[['complexity_of_intents=complex', 'safety_concern=moderate', 'subject_of_expertise=Agriculture'], ['complexity_of_intents=complex', 'format_constraints=1', 'safety_concern=safe', 'subject_of_expertise=Agriculture', 'type_of_in_context_material=1'], ['complexity_of_intents=complex', 'safety_concern=safe', 'type_of_in_context_material=1'], ['complexity_of_intents=simple', 'languages=English', 'subject_of_expertise=Agriculture'], ['complexity_of_intents=simple', 'safety_concern=safe', 'type_of_in_context_material=1'], ['complexity_of_intents=simple', 'format_constraints=1', 'languages=English', 'safety_concern=safe', 'subject_of_expertise=Agriculture'], ['complexity_of_intents=complex', 'languages=English', 'safety_concern=safe', 'type_of_in_context_material=1'], ['complexity_of_intents=complex', 'languages=English', 'safety_concern=safe', 'type_of_in_context_material=1'], ['complexity_of_intents=complex', 'languages=English', 'safety_concern=safe', 'type_of_in_context_material=1'], ['co

So now you have determined 10 feature combinations that seem to work well. The next step is to train RMs and evaluate them.


In [26]:
from beaker import Beaker, ExperimentSpec
from copy import deepcopy

In [28]:
spec = ExperimentSpec.from_file("../../beaker/template.yml")
exp_spec = deepcopy(spec)
template_task = exp_spec.tasks.pop(0)

new_tasks = []
for idx, combination in enumerate(top_combinations):
    feats_to_run = []
    for feat in combination:
        if "min_val" in feat:
            if "token_len_difference" in feat:
                feat = feat.replace("difference", "diff")
            feats_to_run.append(feat.replace("__", "::"))
        else:
            feat_name, value = feat.split("=")
            category = find_meta_category(feat_name)
            if category == "closed_set":
                key = "constraints"
            elif category == "scalar":
                key = "value"
            elif category == "open_set":
                key = "check_for_existence"
            feats_to_run.append(f"{category}::feature_name={feat_name}|{key}={value}")
    # Create beaker task
    task = deepcopy(template_task)
    task.name = f"get-features-datamodel-{idx}"
    task.arguments.extend(["--features"] + feats_to_run)
    new_tasks.append(task)

exp_spec.tasks = new_tasks
exp_spec.validate()
exp_spec.to_file("experiments.yml")

Get finished jobs and download the subsets and create an `experiments.txt` file


In [47]:
# experiment_id = "01J6TS47Q2KNKYRCYHC8A0DE4B"
experiment_id = "01J6WDKDPQCM92REXJ1VCNJ0NW"
top_subsets_dir = Path("top_n_subsets")
top_subsets_dir.mkdir(parents=True, exist_ok=True)
experiments_file = top_subsets_dir / "top_n_subsets_experiments.txt"
beaker = Beaker.from_env("ai2/ljm-oe-adapt")
experiment = beaker.experiment.get(experiment_id)

experiment_ids = []
for job in experiment.jobs:
    if job.is_done:
        # Get output
        dataset_id = job.execution.result.beaker
        beaker.dataset.fetch(
            dataset_id,
            force=True,
            target=top_subsets_dir,
            prefix="data/",
            quiet=True,
        )

        beaker.dataset.fetch(
            dataset_id,
            force=True,
            target=top_subsets_dir,
            prefix="experiments.txt",
            quiet=True,
        )

        with open(top_subsets_dir / "experiments.txt", "r") as f:
            data = f.read().splitlines()
            id = data[0]
            experiment_ids.append(id)

print(experiment_ids)
with open(experiments_file, "a") as f:
    for id in set(experiment_ids):
        f.write(id + "\n")

['human_datamodel_7000_FEATS_c9b064281f0fb06d38fb8e20705fc40b_SWAPS_231::bertscore__min_val-0.67|max_val-1.0___bertscore_length__min_val-0.67|max_val-1.0___scalar__feature_name-complexity_of_intents|value-simple___cosine_sim__min_val-0.33|max_val-0.67___scalar__feature_name-open_endedness|value-no', 'human_datamodel_7000_FEATS_c9b064281f0fb06d38fb8e20705fc40b_SWAPS_231::bertscore__min_val-0.67|max_val-1.0___bertscore_length__min_val-0.67|max_val-1.0___scalar__feature_name-complexity_of_intents|value-simple___cosine_sim__min_val-0.33|max_val-0.67___scalar__feature_name-open_endedness|value-no', 'human_datamodel_7000_FEATS_c9b064281f0fb06d38fb8e20705fc40b_SWAPS_231::bertscore__min_val-0.67|max_val-1.0___bertscore_length__min_val-0.67|max_val-1.0___scalar__feature_name-complexity_of_intents|value-simple___cosine_sim__min_val-0.33|max_val-0.67___scalar__feature_name-open_endedness|value-no', 'human_datamodel_7000_FEATS_32a13a5118bd8537a4d61075e9facc63_SWAPS_250::scalar__feature_name-comple

In [46]:
len(set(experiment_ids))

11